# GINへの同期の失敗を解消する。

GINへの同期で失敗した場合は以下の各セッションをに順次実行してください。<br>
同期が失敗した場合、実行セルの出力結果に`リポジトリ側の変更と競合しました。競合を解決してください。`と表示されます。<br>
それ以外の時は、以下の処理は実行しないでください。

実験フロートップページに戻る場合は[こちら](./experiment.ipynb)。新規タブで開きます。

## 0. 研究リポジトリ名を確認する  
以下のセルを実行すると、この研究実行環境で操作する研究リポジトリ名を確認できます。  

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
utils.show_name('green')

## 1. コンフリクトが発生しているファイルを特定する。

In [ ]:
from utils.common import common
from utils.git import git_module
from IPython.display import clear_output
from utils import conflict_helper_util
import os
from datalad import api
from utils import display_util

conflict_filepaths = git_module.get_conflict_filepaths()
annex_path_list = git_module.get_annex_content_file_paht_list()

# get 'git-annex' conflict path
annex_conflict_filepaths = common.get_AND_elements(conflict_filepaths, annex_path_list)
# get 'git' conflict path
git_conflict_filepaths = list(set(conflict_filepaths) - set(annex_conflict_filepaths))
clear_output()

#auto conflict resolving filepaths
auto_conflict_resolve_filepaths = list[str]()
#custom conflict resolving filepaths
custom_conflict_resolve_filepaths = list[str]()
# setting resolving conflict mode
mode = ''
if len(git_conflict_filepaths) > 0:
    mode = 'git'
    # extract notebook for RF
    auto_conflict_resolve_filepaths, custom_conflict_resolve_filepaths = conflict_helper_util.divide_rf_notebook_or_non_file(git_conflict_filepaths)
    # copy conflicted local RF-notebook to .tmp/conflict
    for path in auto_conflict_resolve_filepaths:
        conflict_helper_util.copy_local_content_to_tmp(path)

annex_rslv_info = dict[str, dict]()
if len(annex_conflict_filepaths) > 0:
    if mode == 'git':
        mode = 'both'
    else:
        mode = 'annex'
        
    os.chdir(os.environ['HOME'])
    os.system('git annex resolvemerge')
    # get informatin for resolving git-annex connfilict
    remote_annex_variant_paths =  git_module.get_remote_annex_variant_path(annex_conflict_filepaths)
    # get information of datalad
    multi_status_info = api.status(path=['experiments/'])
    ## create git-annex information(dict) for resolving conflict
    for conflict_annex_path in annex_conflict_filepaths:
        annex_rslv_info[conflict_annex_path] = dict[str, str]()
        
    to_datalad_get_paths = list[str]()
    for unit_status_info in multi_status_info:
        if unit_status_info['state'] == 'added':
            for conflict_annex_path in annex_conflict_filepaths:
                # adjust file path that is conflicted for extract variant paht
                dirpath = os.path.dirname(conflict_annex_path)
                filename_no_extension = os.path.splitext(os.path.basename(conflict_annex_path))[0]
                target_path = '{}/{}.variant-'.format(dirpath, filename_no_extension)

                path = unit_status_info['path']
                refds = '{}/'.format(unit_status_info['refds'])
                path = path.replace(refds, '', 1)
                if path.startswith(target_path):
                    variant_list = annex_rslv_info[conflict_annex_path]
                    if path in remote_annex_variant_paths:
                        variant_list['remote'] =  path
                        to_datalad_get_paths.append(path)
                    else:
                        variant_list['HEAD'] =  path
                    annex_rslv_info[conflict_annex_path] = variant_list
    # donwloading remote variants content
    api.get(path=to_datalad_get_paths)

# add git content
import os
os.chdir(os.environ['HOME'])
for path in git_conflict_filepaths:
    os.system('git add {}'.format(path))

os.chdir(os.environ['HOME'])
if mode == 'git':
    os.system('git annex lock .')
    os.system('git commit -m "Resolvemerge Result(only git)"')
    os.system('git annex unlock')
elif mode == 'annex':
    os.system('git annex lock .')
    os.system('git commit -m "Resolvemerge Result(only git-annex)"')
    os.system('git annex unlock')
elif mode == 'both':
    os.system('git annex lock .')
    os.system('git commit -m "Resolvemerge Result(git and git-annex)"')
    os.system('git annex unlock')

# Output Informatin to user
if len(git_conflict_filepaths) > 0:
    display_util.display_info(msg='『2. 競合ファイル(git)を修正する。』を必ず実行してください。')
if len(annex_conflict_filepaths) > 0:
    display_util.display_info(msg='『3. 競合ファイル(git-annex)を修正する。』を必ず実行してください。')

In [ ]:
# get conflict variants
# from utils.common import common
# from utils import display_util
# from datalad import api
# import os
# from utils.git import git_module
# from IPython.display import clear_output

# os.chdir(os.environ['HOME'])
# os.system('git annex resolvemerge')

# # get informatin for resolving git-annex connfilict
# os.chdir(os.environ['HOME'])
# remote_annex_variant_paths =  git_module.get_remote_annex_variant_path(annex_conflict_filepaths)

# # get information of datalad
# multi_status_info = api.status(path=['experiments/'])

## create git-annex information(dict) for resolving conflict
# annex_rslv_info = dict[str, dict]()
# for conflict_annex_path in annex_conflict_filepaths:
#     annex_rslv_info[conflict_annex_path] = dict[str, str]()

# to_datalad_get_paths = list[str]()
# for unit_status_info in multi_status_info:
#     if unit_status_info['state'] == 'added':
#         for conflict_annex_path in annex_conflict_filepaths:
#             # adjust file path that is conflicted for extract variant paht
#             dirpath = os.path.dirname(conflict_annex_path)
#             filename_no_extension = os.path.splitext(os.path.basename(conflict_annex_path))[0]
#             target_path = '{}/{}.variant-'.format(dirpath, filename_no_extension)

#             path = unit_status_info['path']
#             refds = '{}/'.format(unit_status_info['refds'])
#             path = path.replace(refds, '', 1)
#             if path.startswith(target_path):
#                 variant_list = annex_rslv_info[conflict_annex_path]
#                 if path in remote_annex_variant_paths:
#                     variant_list['remote'] =  path
#                     to_datalad_get_paths.append(path)
#                 else:
#                     variant_list['HEAD'] =  path
#                 annex_rslv_info[conflict_annex_path] = variant_list

# donwloading remote variants content
# api.get(path=to_datalad_get_paths)




In [ ]:
# # add git content
# from utils.common import common
# import os
# os.chdir(os.environ['HOME'])
# for path in git_conflict_filepaths:
#     os.system('git add {}'.format(path))

In [ ]:
# import os
# os.chdir(os.environ['HOME'])
# if mode == 'git':
#     os.system('git annex lock .')
#     os.system('git commit -m "Resolvemerge Result(only git)"')
#     os.system('git annex unlock')
# elif mode == 'annex':
#     os.system('git annex lock .')
#     os.system('git commit -m "Resolvemerge Result(only git-annex)"')
#     os.system('git annex unlock')
# elif mode == 'both':
#     os.system('git annex lock .')
#     os.system('git commit -m "Resolvemerge Result(git and gi-annex)"')
#     os.system('git annex unlock')

In [ ]:
# # Output Informatin to user
# if len(git_conflict_filepaths) > 0:
#     display_util.display_info(msg='『2. 競合ファイル(git)を修正する。』を必ず実行してください。')
# if len(annex_conflict_filepaths) > 0:
#     display_util.display_info(msg='『3. 競合ファイル(git-annex)を修正する。』を必ず実行してください。')

## 2. 競合ファイル(git)を修正する。

2-1.~2-3の説明をよく読み、実行してください。<br>

### 2-1. 『2. 競合ファイル(git)を修正する。』タスクを有効化する。

In [ ]:
from utils import display_util
activate_git_rslv = False
if mode == 'annex':
    activate_git_rslv = False
    display_util.display_warm(msg='『2. 競合ファイル(git)を修正する。』は実行する必要ありません。')
elif mode == 'git' or mode == 'both':
    activate_git_rslv = True
    display_util.display_info(msg='有効化されました。次のセルにお進みください。')
else:
    display_util.display_err(msg='『1. コンフリクトが発生しているファイルを特定する。』が実行されていない可能性があります')

### 2-2. 競合ファイルのパスの確認と修正の確定を行う。

以下の手順に従って競合ファイルを修正してください。<br>

1. 下のセルを実行する。
2. 競合ファイルのパスを確認する。
   
※ 競合が発生したファイルパスと`修正完了`ボタン(灰色)が表示されます。<br>
※ この時点では、`修正完了`ボタンをクリックしないください。

![競合ファイルパス](./images/Display_of_conflicting_file_paths.png)

3. ダッシュボートの`File` > `Open...`と順にクリックする。
4. 競合が発生しているファイルにチェックを入れ、`edit`ボタンをクリックする。
5. ファイルを編集して競合を解消する。

* 編集前

![編集前](./images/Before_conflict_resolution.png)

* 編集後

![編集前](./images/After_conflict_resolution.png)

6. ダッシュボートの`File` > `Save`と順にクリックし、保存する。
7. `修正完了`ボタンをクリックする。（ボタンがクリックされる緑に点灯します）
8. 全ての`修正完了`ボタンが緑になるまで、上記の2.~7.を繰り返す。

In [ ]:
from utils import conflict_helper_util
from ipywidgets import Button
from utils import display_util
from IPython.display import display

if activate_git_rslv:
    # Overwrite conflicted notebook form .tmp to working directory
    for path in auto_conflict_resolve_filepaths:
        conflict_helper_util.copy_tmp_to_working(path)
    
    # Display resolving file paths by user
    if len(custom_conflict_resolve_filepaths) > 0:
        def on_click_callback(clicked_button: Button) -> None:
            clicked_button.button_style = 'success'
        
        display_util.display_msg(msg='競合が発生しているファイルは以下です。', fore=None, back=None, tag='p')
        display_util.display_msg(msg='全ての修正完了ボタンが緑に点灯したら次にお進みください。', fore=None, back=None, tag='p')
        for i, git_conflict_filepath in enumerate(custom_conflict_resolve_filepaths):
            display_util.display_msg(msg='{} : {}'.format(int(i) + 1, git_conflict_filepath), fore=None, back=None, tag='p')
            button = Button(description='修正完了', button_style='')
            button.on_click(on_click_callback)
            display(button)
    else:
        display_util.display_info(msg="次にお進みください。")
else:
    display_util.display_warm(msg='『2. 競合ファイル(git)を修正する。』は実行する必要ありません。')


In [ ]:
# Overwrite conflicted notebook form .tmp to working directory
# from utils import conflict_helper_util
# for path in auto_conflict_resolve_filepaths:
#     conflict_helper_util.copy_tmp_to_working(path)

In [ ]:
# ユーザにgit管理ファイルを手動で編集してもらう。
# from ipywidgets import Button
# from utils import display_util
# from IPython.display import display

# def on_click_callback(clicked_button: Button) -> None:
#     clicked_button.button_style = 'success'
#     display_util.display_info(msg="次にお進みください。")

# display_util.display_msg(msg='競合が発生しているファイルは以下です。', fore=None, back=None, tag='p')
# for i, git_conflict_filepath in enumerate(git_conflict_filepaths):
#     display_util.display_msg(msg='{} : {}'.format(int(i) + 1, git_conflict_filepath), fore=None, back=None, tag='p')
#     button = Button(description='修正完了', button_style='')
#     button.on_click(on_click_callback)
#     display(button)


### 2-3. 更新内容を記録する。

In [ ]:
import os
if activate_git_rslv:
    os.chdir(os.environ['HOME'])
    # crete path list for git add
    git_add_path = git_conflict_filepaths + auto_conflict_resolve_filepaths
    #git add
    for path in git_add_path:
        os.system('git add {}'.format(path))
    # git annex lock to prevent annex content from being committed
    os.system('git annex lock .')
    # git commit for git conflicted file
    os.system('git commit -m "conflict resolution(git)"')
    # git annex unlock to allow modification of annex content
    os.system('git annex unlock')
else:
    display_util.display_warm(msg='『2. 競合ファイル(git)を修正する。』は実行する必要ありません。')

In [ ]:
# # add git content
# from utils.common import common
# import os
# os.chdir(os.environ['HOME'])
# # crete path list for git add
# git_add_path = git_conflict_filepaths + auto_conflict_resolve_filepaths

# for path in git_add_path:
#     stdout, stderr, rt = common.exec_subprocess('git add {}'.format(path), False)
#     result = stdout.decode('utf-8')

In [ ]:
# import os
# os.chdir(os.environ['HOME'])
# if mode == 'both' or mode == 'git':
#     os.system('git annex lock .')
#     os.system('git commit -m "conflict resolution(git)"')
#     os.system('git annex unlock')

## 3. 競合ファイル(git-annex)を修正する。

3-1.~3-5の説明をよく読み、実行してください。<br>


### 3-1. 『3. 競合ファイル(git-annex)を修正する。』タスクを有効化する。

In [ ]:
from utils import display_util
activate_annex_rslv = False
if mode == 'git':
    activate_annex_rslv = False
    display_util.display_info(msg='『3. 競合ファイル(git-annex)を修正する。』は実行する必要ありません。')
elif mode == 'annex' or mode == 'both':
    activate_annex_rslv = True
    display_util.display_info(msg='有効化されました。次のセルにお進みください。')
else:
    display_util.display_err(msg='『1. コンフリクトが発生しているファイルを特定する。』が実行されていない可能性があります')


### 3-2. 競合ファイルに対するアクションを選択する。

競合が発生したファイルに対して、HEAD(ローカル)のバージョン(HEADバリアント)とRemote先のバージョン(Remoteバリアント)のデータが複製されています。<br>
ここでは、競合が発生にしたファイルの内容を確認した上で、いずれかのアクションを選択してください。<br>

選択できるアクションについて
1. `HEADのファイルを残す` : HEADバリアントを元のファイル名で保存します。
2. `Remoteのファイルを残す` : Remoteバリアントを元のファイル名で保存します。
3. `両方残す` : HEADバリアントおよびRemoteバリアントのいずれも保存する。(ファイル名は次の『3-3. ≪両方を残す≫が選択されたファイルに名前をつける。』で変更することできます)


競合が発生した各ファイルについて以下の情報と選択フォームが表示されます。
* 競合が発生ファイルパス
* HEADバリアントのファイルパス
* Remoteバリアントのファイルパス
* アクションの選択欄

(例)
[アクション選択フォーム]()

In [ ]:
import panel as pn
from ipywidgets import Button
from utils import display_util, conflict_helper_util
from IPython.display import display

if activate_annex_rslv:
    pn.extension()
    display_util.display_msg(msg='各ファイルに対するアクションを選択し、確定してください。', fore=None, back=None, tag='p')
    column = pn.Column()
    form_list = list()
    options = conflict_helper_util.get_annex_conflict_options()
    index = 1
    for conflict_filepath, value in sorted(annex_rslv_info.items()):
        title = pn.widgets.StaticText(name=str(index), value=conflict_filepath, width=700)
        form_list.append(title)
        head = pn.widgets.StaticText(name='HEAD', value=value['HEAD'], width=700)
        form_list.append(head)
        remote = pn.widgets.StaticText(name='Remote', value=value['remote'], width=700)
        form_list.append(remote)
        form = pn.widgets.Select(options=options, width=200)
        form_list.append(form)
        index+=1

    # Display GUI.
    for form in form_list:
        column.append(form)
    display(column)

    def on_click_callback(clicked_button: Button) -> None:
        clicked_button.button_style = 'success'
        display_util.display_msg(msg='選択された内容は以下になります。お間違いが無ければ次にお進みください。', fore=None, back=None, tag='p')
        display_util.display_msg(msg='もし、変更したい場合は、再度、セルを実行してください。', fore=None, back=None, tag='p')
        total = len(form_list)
        block_num = int(total/4)
        for i in range(block_num):
            start_index =i*4
            end_index = start_index+4
            block = form_list[start_index:end_index]
            v = annex_rslv_info[block[0].value]
            v['action'] = block[3].value
            annex_rslv_info[block[0].value] = v
            display_util.display_msg(msg='{}. {} : {}'.format(i+1, block[0].value, block[3].value), fore=None, back=None, tag='p')

    button = Button(description='確定', button_style='')
    button.on_click(on_click_callback)
    display(button)
else:
    display_util.display_info(msg='『3-1. 競合ファイルに対するアクションを選択する。』は実行する必要ありません。')

### 3-3. ≪両方を残す≫が選択されたファイルに名前をつける。

『3-2. 競合ファイルに対するアクションを選択する。』で`両方残す`が選択されたファイルに対して保存するファイル名を入力してください。<br>
（注 1）元のファイルを同じ名前の場合、同名コンテンツのgit履歴を引継ぎますが、別名のファイル名の場合、新規コンテンツとしてgit履歴に記録されます。<br>
（注 2）入力フォームにはファイル名のみ入力してください。<br>
（注 3）既に存在するファイル名は指定できません。<br>
（注 4）重複してファイル名を指定できません。<br>
（注 5）現在の各バリアント名はそのままファイル名として指定できません。<br>

対象の各ファイルについて以下の情報と入力フォームが表示されます。
* 競合が発生ファイルパス
* HEADバリアントのファイルパス
  * ファイル名の入力フォーム
* Remoteバリアントのファイルパス
  * ファイル名の入力フォーム




In [ ]:
from utils import conflict_helper_util, display_util
import os
import panel as pn

if activate_annex_rslv:
    pn.extension()
    ## 両方残す場合、ファイル名の変更を促す
    column = pn.Column()
    rename_form_list = list()
    if conflict_helper_util.is_more_than_both_remain(annex_rslv_info):
        display_util.display_msg(msg='<<{}>>を選択されたファイルに対して新しいファイル名を入力してください。'.format(conflict_helper_util.get_value_BOTH_REMAIN()), fore=None, back=None, tag='p')
        display_util.display_msg(msg='ファイル名のみ入力することに注意してください。'.format(conflict_helper_util.get_value_BOTH_REMAIN()), fore=None, back=None, tag='p')
        index = 1
        for conflict_filepath, value in sorted(annex_rslv_info.items()):
            if conflict_helper_util.is_both_remain(annex_rslv_info[conflict_filepath]):
                # ファイル名変更フォームの生成
                title = pn.widgets.StaticText(name=str(index), value=conflict_filepath, width=700)
                rename_form_list.append(title)
                head = pn.widgets.StaticText(name='HEAD', value=value['HEAD'], width=700)
                rename_form_list.append(head)
                rename_input_for_head = pn.widgets.TextInput(name=os.path.dirname(value['HEAD']), placeholder=os.path.basename(value['HEAD']), width=700)
                rename_form_list.append(rename_input_for_head)
                remote = pn.widgets.StaticText(name='Remote', value=value['remote'], width=700)
                rename_form_list.append(remote)
                rename_input_for_remote = pn.widgets.TextInput(name=os.path.dirname(value['remote'])placeholder=os.path.basename(value['remote']), width=700)
                rename_form_list.append(rename_input_for_remote)
                index +=1

        # Display GUI.
        for form in rename_form_list:
            column.append(form)
        
        def on_click_callback(event):
            total = len(rename_form_list)
            block_num = int(total/5)
            
            for i in range(block_num):
                start_index =i*5
                end_index = start_index+5
                block = rename_form_list[start_index:end_index]
                v = annex_rslv_info[block[0].value]
                rename_dict = dict()
                rename_dict[block[1].value] = block[2].value_input
                rename_dict[block[3].value] = block[4].value_input
                v['rename'] = rename_dict
                annex_rslv_info[block[0].value] = v
            # TODO : verify file name
            duplicates_paths, invalid_names, existence_file_paths, has_slash_file_names = conflict_helper_util.verify_resolve_file_name(annex_rslv_info)
            if len(duplicates_paths)>0 or len(invalid_names)>0 or len(existence_file_paths)>0 or len(has_slash_file_names):
                button.button_type = "danger"
                display_util.display_err(msg='不正な値が入力されました。再度、セルを実行し正しいファイル名をしてください')
                if len(invalid_names)>0:
                    display_util.display_err(msg='バリアント名を指定されています。以下のバリアント名は指定しないでください。')
                    for keep_vatiant_name_file_path in invalid_names:
                        display_util.display_msg(msg='・{}'.format(keep_vatiant_name_file_path), fore=None, back=None, tag='p')
                
                if len(duplicates_paths)>0:
                    display_util.display_err(msg='重複しているファイルがあります。')
                    for duplicates_path in duplicates_paths:
                        display_util.display_msg(msg='・{}'.format(duplicates_path), fore=None, back=None, tag='p')
                if len(existence_file_paths)>0:
                    display_util.display_err(msg='既に存在しているファイル名で指定されました。')
                    for existence_file_path in existence_file_paths:
                        display_util.display_msg(msg='・{}'.format(existence_file_path), fore=None, back=None, tag='p')
                if len(has_slash_file_names)>0:
                    display_util.display_err(msg='ファイル名のみ入力してください。(ディレクトリの変更はできません)')
                    for has_slash_file_name in has_slash_file_names:
                        display_util.display_msg(msg='・{}'.format(has_slash_file_name), fore=None, back=None, tag='p')

            else:
                button.button_type = "success"
                button.name = "ファイル名が確定されました。次の処理にお進みください。"
        
        button = pn.widgets.Button(name= "確定", button_type= "primary")
        button.on_click(on_click_callback)
        column.append(button)
        display(column)
    else:
        display_util.display_info(msg='次にお進みください')
else:
    display_util.display_info(msg='『3-2. ≪両方を残す≫が選択されたファイルに名前をつける。』は実行する必要ありません。')

### 3-4. 選択させたアクションに従って、データの調整を行う。

In [ ]:
# adjust conflicted git-annex content
from utils import conflict_helper_util, display_util
from utils.git import git_module
os.chdir('/home/jovyan/WORKFLOWS/')
from except_class import rename_err, action_err

git_annex_target_path = list[str]()
delete_filepaths = list[str]()
if activate_annex_rslv:
    try:
        if conflict_helper_util.check_annex_resolve_info(annex_rslv_info):
            for i, k in enumerate(annex_rslv_info):
                unit_info = annex_rslv_info[k]
                action = unit_info['action']
                if action == conflict_helper_util.get_value_BOTH_REMAIN():
                    # remain both annex variants
                    rename_info = unit_info['rename']
                    for i, k in enumerate(rename_info):
                        future_name= rename_info[k]
                        new_path = conflict_helper_util.rename_file(k, future_name)
                        git_annex_target_path.append(new_path)
                elif action == conflict_helper_util.get_value_REMOTE_REMAIN():
                    # remain remote contents

                    # delete HEAD variants
                    head_file_path = unit_info['HEAD']
                    conflict_helper_util.delete_file(head_file_path)

                    # rename Remote variants by base file name
                    remote_file_path = unit_info['remote']
                    new_path = conflict_helper_util.rename_file(remote_file_path, os.path.basename(k))
                    git_annex_target_path.append(new_path)

                elif action == conflict_helper_util.get_value_HEAD_REMAIN():
                    # remain HEAD content
                    # rename HEAD variants by base file name
                    head_file_path = unit_info['HEAD']
                    new_path = conflict_helper_util.rename_file(head_file_path, os.path.basename(k))
                    git_annex_target_path.append(new_path)
                    # delete Remote variants
                    remote_file_path = unit_info['remote']
                    conflict_helper_util.delete_file(remote_file_path)
                else:
                    raise Exception('不正な操作が行われました。')
            # get delete filepath in resolving annex confilct
            delete_filepaths = git_module.get_delete_filepaths()
    except rename_err.RenameError:
        display_util.display_err(msg='『3-2. ≪両方を残す≫が選択されたファイルに名前をつける』から再実行してください。')
    except action_err.ActionError:
        display_util.display_err(msg='『3-1. 競合ファイルに対するアクションを選択する。』から再実行してください。')
else:
    display_util.display_info(msg='『3-3. 選択させたアクションに従って、データの調整を行う。』は実行する必要ありません。')

### 3-5. 更新内容を記録する。

In [ ]:
# from utils.git import git_module

# # get delete filepath in resolving annex confilct
# delete_filepaths = git_module.get_delete_filepaths()

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils

os.chdir(os.environ['HOME'])
if activate_git_rslv:
    # git add for delete file
    for path in delete_filepaths:
        os.system('git add {}'.format(path))
    
    # git annex add for resolved content
    for target_path in git_annex_target_path:
        os.system('git annex add {}'.format(target_path))
    
    # add metadata for annex content
    for target_path in git_annex_target_path:
        utils.register_metadata_for_annexdata(target_path)
    
    os.system('git annex lock .')
    os.system('git commit -m "conflict resolution (git-annex)"')


In [ ]:
# # add process
# from utils.common import common
# import os

# ## git
# os.chdir(os.environ['HOME'])
# for path in delete_filepaths:
#     stdout, stderr, rt = common.exec_subprocess('git add {}'.format(path), False)
#     result = stdout.decode('utf-8')

# ## git-annex
# os.chdir(os.environ['HOME'])
# for target_path in git_annex_target_path:
#     stdout, stderr, rt = common.exec_subprocess('git annex add {}'.format(target_path), False)
#     result = stdout.decode('utf-8')

In [ ]:
# add metadata for annex content
# os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
# from util.scripts import utils
# os.chdir(os.environ['HOME'])
# for target_path in git_annex_target_path:
#     utils.register_metadata_for_annexdata(target_path)

In [ ]:
# from utils.common import common
# import os
# os.chdir(os.environ['HOME'])
# os.system('git annex lock .')
# os.system('git commit -m "conflict resolution (git-annex)"')

## 4. GINへ修正内容をプッシュする。

In [ ]:
# git add process(notebook)
import os
from IPython.display import display, Javascript
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
from IPython.display import clear_output
import traceback

display(Javascript('IPython.notebook.save_checkpoint();'))
notebookpath = 'WORKFLOWS/conflict_helper.ipynb'
os.system('git add {}'.format(notebookpath))
os.system('git annex lock .')
commit_msg = 'Push Your Resolving Log for conflict to Remote'
os.system('git commit -m "{}"'.format(commit_msg))

# Push to GIN
try:
    os.chdir(os.environ['HOME'])
    utils.push()
    os.system('git annex unlock')
    # clear_output()
except:
    datalad_error = traceback.format_exc()
    display_util.display_err(msg='[ERROR] : '.format(datalad_error))
    display_util.display_err(msg='GIN-forkへのコミットのプッシュが失敗しました。')
else:
    display_util.display_info(msg='GIN-forkへのコミットのプッシュが成功しました。\n以上で競合解消タスクは完了です。')


In [ ]:
# # add process
# from utils.common import common
# import os
# from IPython.display import display, Javascript
# display(Javascript('IPython.notebook.save_checkpoint();'))
# ## git
# os.chdir(os.environ['HOME'])
# notebookpath = 'WORKFLOWS/conflict_helper.ipynb'
# stdout, stderr, rt = common.exec_subprocess('git add {}'.format(notebookpath), False)
# result = stdout.decode('utf-8')

In [ ]:
# from utils.common import common
# import os
# os.chdir(os.environ['HOME'])
# !git annex lock .
# commit_msg = 'Push Your Resolving Log for conflict to Remote'
# stdout, stderr, rt = common.exec_subprocess('git commit -m "{}"'.format(commit_msg), False)
# result = stdout.decode('utf-8')


In [ ]:
# import os
# os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
# from util.scripts import utils
# from IPython.display import clear_output
# import traceback
# # Push to GIN
# try:
#     os.chdir(os.environ['HOME'])
#     utils.push()
#     os.system('git annex unlock')
#     # clear_output()
# except:
#     datalad_error = traceback.format_exc()
#     display_util.display_err(msg='[ERROR] : '.format(datalad_error))
#     display_util.display_err(msg='GIN-forkへのコミットのプッシュが失敗しました。')
# else:
#     display_util.display_info(msg='GIN-forkへのコミットのプッシュが成功しました。\n以上で競合解消タスクは完了です。')
    

## 5. 実験フロートップページに遷移する

以下のセルを実行し、表示されるリンクをクリックして実験フロートップページに戻ってください。 

In [ ]:
from IPython.display import display, HTML, Javascript
display(HTML("<a href='./experiment.ipynb'>実験フロートップページに遷移する</a>"))
display(Javascript('IPython.notebook.save_checkpoint();'))